In [1]:
PROMPT = {}

In [23]:
PROMPT['evaluation_from_test']= '''
Bạn là một chuyên gia luật, được giao nhiệm vụ là phân tích đánh giá câu trả lời của một sinh viên so với đáp án đúng (standard golden answer) về hỏi đáp luật.

## Các tiêu chí đánh giá:
Bạn sẽ phải so sánh câu trả lời với đáp án để cho điểm số từ 0-10 điểm, trong đó:
### 0 điểm: không có câu trả lời được cung cấp
### 1-2 điểm: Đã có được một số thông tin được cung cấp, nhưng nó hoàn toàn không liên quan, trùng khớp với câu trả lời đúng.
### 3-5 điểm: Đã trả lời được một vài thông tin chính xác hữu ích, có trong câu trả lời đúng, tuy nhiên chúng chưa đầy đủ và chứa nhiều thông tin không chính xác, liên quan.
### 6-7 điểm: Đã trả lời được nhiều thông tin có ích hơn khoảng 50% thông tin có trong câu trả lời đúng. Tuy nhiên trả lời vẫn dài dòng, ít thông tin không chính xác, không liên quan hơn.
### 8-9 điểm: Đã trả lời được nhiều thông tin có ích hơn khoảng 70%-90% thông tin có trong câu trả lời đúng, những thông tin không liên quan có ít hơn nhưng nó không quá ảnh hưởng đến câu trả lời
### 10 điểm: Chứa tất các nội dung được nhắc đến trong câu trả lời đúng, không thừa không thiếu, mặc dù có thể diễn đạt theo cách khác.

## Đầu ra dạng JSON:
```json
{
  "Score" : "<điểm số>" # chỉ cần ghi mỗi số
  "Explain" : "<giải thích vì sao đạt điểm đó>"
}
```

## Câu hỏi:
<<q>>

## Đây là câu trả lời đúng:
<<sga>>

## Đây là câu trả lời mà bạn cần đánh giá
<<a>>
'''

In [70]:
PROMPT['evaluation_from_test_kw']= '''
Bạn là một chuyên gia luật, được giao nhiệm vụ là phân tích đánh giá câu trả lời của một sinh viên so với đáp án đúng (standard golden answer) về hỏi đáp luật.

## Các TIÊU CHÍ đánh giá:
Bạn sẽ phải trích xuất các từ khóa, thông tin quan trọng của câu trả lời với so với đáp án đúng để cho điểm số từ 0-10 điểm, trong đó:

### 0-2 điểm: Không có một từ khóa quan trọng được trích xuất ra
### 3-5 điểm: Đã có một vài từ khóa, thông tin quan trọng. Tuy nhiên vẫn chứa nhiều thông tin sai.
### 6-7 điểm: Đã có một vài từ khóa, thông tin quan trọng khoảng 50%. Chứa ít thông tin sai hơn.
### 8-9 điểm: Đã có một vài từ khóa, thông tin quan trọng khoảng 60%-90% thông tin có trong câu trả lời đúng, những thông tin không liên quan có ít hơn nhưng nó không quá ảnh hưởng đến câu trả lời
### 10 điểm: Chứa tất các nội dung được nhắc đến trong câu trả lời đúng, không thừa không thiếu, mặc dù có thể diễn đạt theo cách khác.

LƯU Ý: Cho dù câu trả lời có dài hay ngắn hơn đáp án thì điều đó không quan trọng, miễn là trả lời đúng ý.

## Đầu ra dạng JSON:
```json
{
  "sga" : ["<từ khóa quan trọng>", "<thông tin quan trọng>", ...] # Thông tin quan trọng của đáp án đúng
  "anwser" : ["<từ khóa quan trọng>", "<thông tin quan trọng>", ...] # Thông tin quan trọng của câu trả lời
  "Explain" : "<phân tích thông tin quan trọng ở trên, so sánh, đánh giá câu trả lời với đáp án dựa trên TIÊU CHÍ đã đề cập>"
  "Score" : "<điểm số>" # chỉ cần ghi mỗi số
}
```

## Câu hỏi:
<<q>>

## Đây là câu trả lời đúng (sga):
<<sga>>

## Đây là câu trả lời mà bạn cần đánh giá (answer)
<<a>>
'''

In [2]:
!pip install openai
!pip install python-dotenv

In [8]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import json
import time

# Load file .env
load_dotenv()

False

In [4]:
client = OpenAI(api_key="")

In [9]:
def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2, # this is the degree of randomness of the model's output
    )
    print(model)
    return response.choices[0].message.content

In [10]:
def get_valid_json_gpt40mini(prompt,max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        print(f"attempt : {attempts}")
        HEAD = """In your previous response you responded with the wrong request format below, now you have to do it again and respond with the correct JSON as described below"""

        if attempts != 0:
            prompt = HEAD + "\n" + prompt

        res_second = get_completion(prompt)  # Gọi LLM
        try:
            dlist = json.loads(res_second.strip("```").strip("json").strip("\n```"))
            print("Định dạng JSON hợp lệ!")
            return dlist
        except json.JSONDecodeError:
            attempts += 1
            print(f"JSON không hợp lệ, thử lại ({attempts}/{max_attempts})...")
    return "Not"

In [11]:
import json
with open('/content/TULUAN_HYBRID.json', 'r', encoding='utf-8') as f:
    TULUAN_HYBRID = json.load(f)

In [12]:
len(TULUAN_HYBRID)

28

In [17]:
TULUAN_HYBRID[0]

{'q': 'Theo Luật hôn nhân và gia đình năm 2014, những nguyên tắc cơ bản của chế độ hôn nhân và gia đình ở nước ta được quy định như thế nào?',
 'hybrid': '## Nguyên Tắc Cơ Bản của Chế Độ Hôn Nhân và Gia Đình theo Luật Hôn Nhân và Gia Đình Năm 2014\n\nLuật Hôn nhân và Gia đình năm 2014, có hiệu lực từ ngày 01/01/2015, đã quy định một cách rõ ràng các nguyên tắc cơ bản trong chế độ hôn nhân và gia đình tại Việt Nam. Những nguyên tắc này không chỉ bảo đảm quyền và nghĩa vụ của các bên trong quan hệ hôn nhân mà còn thể hiện sự bình đẳng và công bằng giữa các thành viên trong gia đình. Dưới đây là một số nguyên tắc cơ bản được quy định trong bộ luật này.\n\n### 1. Nguyên Tắc Một Vợ Một Chồng\n\nLuật quy định rằng chế độ hôn nhân tại Việt Nam là hôn nhân một vợ một chồng. Điều này nhấn mạnh rằng mỗi người chỉ có thể kết hôn với một người khác tại cùng một thời điểm, nhằm bảo vệ quyền lợi hợp pháp và đạo đức trong quan hệ hôn nhân.\n\n### 2. Nguyên Tắc Bình Đẳng Giữa Vợ và Chồng\n\nLuật Hôn n

In [14]:
with open('/content/TULUAN_NAIVE.json', 'r', encoding='utf-8') as f:
    TULUAN_NAIVE = json.load(f)

In [15]:
len(TULUAN_NAIVE)

28

In [16]:
TULUAN_NAIVE[0]

{'q': 'Theo Luật hôn nhân và gia đình năm 2014, những nguyên tắc cơ bản của chế độ hôn nhân và gia đình ở nước ta được quy định như thế nào?',
 'naive': 'Theo Luật Hôn nhân và Gia đình năm 2014, các nguyên tắc cơ bản của chế độ hôn nhân và gia đình ở Việt Nam được quy định tại Điều 2. Những nguyên tắc này nhằm đảm bảo sự phát triển văn hóa và xã hội trong bối cảnh của đất nước, và cũng bảo vệ quyền lợi và nghĩa vụ của các thành viên trong gia đình. Dưới đây là các nguyên tắc cụ thể:\n\n1. **Hôn nhân tự nguyện và bình đẳng**: Hôn nhân phải được xây dựng trên cơ sở tự nguyện của cả hai bên, không chỉ yêu cầu sự đồng ý mà còn phải đảm bảo sự bình đẳng giữa vợ và chồng. Nguyên tắc này nhấn mạnh rằng không có ai bị ép buộc kết hôn và cả hai vợ chồng đều có quyền và nghĩa vụ ngang nhau trong gia đình.\n\n2. **Sự tôn trọng sự đa dạng**: Luật pháp bảo vệ hôn nhân giữa các công dân thuộc các dân tộc, tôn giáo khác nhau, cũng như giữa người theo tôn giáo và người không theo tôn giáo. Sự đa dạng 

In [18]:
with open('/content/TUVAN_HYBRID_ver2.json', 'r', encoding='utf-8') as f:
    TUVAN_HYBRID = json.load(f)

In [19]:
len(TUVAN_HYBRID)

14

In [24]:
TUVAN_HYBRID[0]

{'q': 'Vợ chồng tôi lấy nhau đã nhiều năm nhưng vẫn chưa có con. Năm 2015, vợ chồng tôi quyết định nhận con nuôi và đã thực hiện theo đúng quy định về việc nhận con nuôi. Đến nay, tôi được biết vợ chồng tôi có thể nhờ mang thai hộ. Vậy xin hỏi, vợ chồng tôi đã nhận con nuôi thì có nhờ người khác mang thai hộ được không?',
 'hybrid': '### Vấn đề mang thai hộ sau khi nhận con nuôi\n\nViệc mang thai hộ trong trường hợp vợ chồng đã nhận con nuôi là một vấn đề pháp lý phức tạp, liên quan đến các quy định trong Luật Hôn nhân và Gia Đình, cũng như Luật Nuôi con nuôi tại Việt Nam. Dưới đây là điểm trả lời cụ thể cho câu hỏi của bạn.\n\n#### 1. Quy định về mang thai hộ \n\nTheo quy định tại **Luật Hôn Nhân và Gia Đình năm 2014** (số 52/2014/QH13), việc mang thai hộ được điều chỉnh tại Điều 95. Điều luật này quy định rằng mang thai hộ vì mục đích nhân đạo được áp dụng cho các cặp vợ chồng không thể sinh con tự nhiên. Như vậy, trong trường hợp của bạn, với Điều 95, vợ chồng bạn có thể thực hiện v

In [20]:
with open('/content/TUVAN_NAIVE_ver2.json', 'r', encoding='utf-8') as f:
    TUVAN_NAIVE = json.load(f)

In [21]:
len(TUVAN_NAIVE)

14

In [31]:
test = PROMPT['evaluation_from_test'].replace("<<q>>", TULUAN_NAIVE[0]['q']).replace("<<sga>>", TULUAN_NAIVE[0]['a']).replace("<<a>>", TULUAN_NAIVE[0]['naive'])

In [32]:
print(test)


Bạn là một chuyên gia luật, được giao nhiệm vụ là phân tích đánh giá câu trả lời của một sinh viên so với đáp án đúng (standard golden answer) về hỏi đáp luật.

## Các tiêu chí đánh giá:
Bạn sẽ phải so sánh câu trả lời với đáp án để cho điểm số từ 0-10 điểm, trong đó:
### 0 điểm: không có câu trả lời được cung cấp
### 1-2 điểm: Đã có được một số thông tin được cung cấp, nhưng nó hoàn toàn không liên quan, trùng khớp với câu trả lời đúng.
### 3-5 điểm: Đã trả lời được một vài thông tin chính xác hữu ích, có trong câu trả lời đúng, tuy nhiên chúng chưa đầy đủ và chứa nhiều thông tin không chính xác, liên quan.
### 6-7 điểm: Đã trả lời được nhiều thông tin có ích hơn khoảng 50% thông tin có trong câu trả lời đúng. Tuy nhiên trả lời vẫn dài dòng, ít thông tin không chính xác, không liên quan hơn.
### 8-9 điểm: Đã trả lời được nhiều thông tin có ích hơn khoảng 70%-90% thông tin có trong câu trả lời đúng, những thông tin không liên quan có ít hơn nhưng nó không quá ảnh hưởng đến câu trả lời

In [35]:
res = get_valid_json_gpt40mini(test)

attempt : 0
gpt-4o-mini
Định dạng JSON hợp lệ!


In [29]:
res

{'Score': '8',
 'Explain': 'Câu trả lời đã cung cấp nhiều thông tin hữu ích và có liên quan đến nguyên tắc cơ bản của chế độ hôn nhân và gia đình theo Luật Hôn nhân và Gia đình năm 2014. Tuy nhiên, một số thông tin chưa hoàn toàn đầy đủ và không hoàn toàn khớp với nội dung của đáp án đúng, ví dụ như việc không đề cập đến các nguyên tắc cụ thể như trách nhiệm bảo vệ trẻ em, người cao tuổi, và kế thừa văn hóa dân tộc. Mặc dù có sự diễn đạt tốt và rõ ràng, nhưng vẫn thiếu một số điểm quan trọng trong nội dung chính của câu trả lời đúng.'}

In [36]:
res

{'Score': 9,
 'Explain': 'Câu trả lời đã cung cấp khoảng 80-90% thông tin có ích từ đáp án đúng, bao gồm các nguyên tắc cơ bản của chế độ hôn nhân và gia đình theo Luật Hôn nhân và Gia đình năm 2014. Tuy nhiên, một số chi tiết như việc kế thừa và phát huy truyền thống văn hóa, đạo đức tốt đẹp của dân tộc Việt Nam không được đề cập rõ ràng. Câu trả lời cũng có phần diễn đạt dài dòng nhưng không ảnh hưởng lớn đến nội dung chính.'}

# TULUAN_HYBRID

In [37]:
TULUAN_HYBRID[0]

{'q': 'Theo Luật hôn nhân và gia đình năm 2014, những nguyên tắc cơ bản của chế độ hôn nhân và gia đình ở nước ta được quy định như thế nào?',
 'hybrid': '## Nguyên Tắc Cơ Bản của Chế Độ Hôn Nhân và Gia Đình theo Luật Hôn Nhân và Gia Đình Năm 2014\n\nLuật Hôn nhân và Gia đình năm 2014, có hiệu lực từ ngày 01/01/2015, đã quy định một cách rõ ràng các nguyên tắc cơ bản trong chế độ hôn nhân và gia đình tại Việt Nam. Những nguyên tắc này không chỉ bảo đảm quyền và nghĩa vụ của các bên trong quan hệ hôn nhân mà còn thể hiện sự bình đẳng và công bằng giữa các thành viên trong gia đình. Dưới đây là một số nguyên tắc cơ bản được quy định trong bộ luật này.\n\n### 1. Nguyên Tắc Một Vợ Một Chồng\n\nLuật quy định rằng chế độ hôn nhân tại Việt Nam là hôn nhân một vợ một chồng. Điều này nhấn mạnh rằng mỗi người chỉ có thể kết hôn với một người khác tại cùng một thời điểm, nhằm bảo vệ quyền lợi hợp pháp và đạo đức trong quan hệ hôn nhân.\n\n### 2. Nguyên Tắc Bình Đẳng Giữa Vợ và Chồng\n\nLuật Hôn n

In [71]:
RES_TULUAN_HYBRID = []

In [73]:
RES_TULUAN_HYBRID

[]

In [74]:
for i in range(len(TULUAN_HYBRID)):

    test = PROMPT['evaluation_from_test_kw'].replace("<<q>>", TULUAN_HYBRID[i]['q']).replace("<<sga>>", TULUAN_HYBRID[i]['a']).replace("<<a>>", TULUAN_HYBRID[i]['hybrid'])

    res = get_valid_json_gpt40mini(test)

    print(res)

    RES_TULUAN_HYBRID.append(res)


attempt : 0
gpt-4o-mini
Định dạng JSON hợp lệ!
{'sga': ['nguyên tắc cơ bản', 'hôn nhân tự nguyện', 'một vợ một chồng', 'bình đẳng giữa vợ và chồng', 'quyền và nghĩa vụ giữa cha mẹ và con', 'không phân biệt đối xử', 'trách nhiệm bảo vệ trẻ em', 'kế thừa truyền thống văn hóa'], 'anwser': ['Nguyên Tắc Một Vợ Một Chồng', 'Nguyên Tắc Bình Đẳng Giữa Vợ và Chồng', 'Quyền và Nghĩa Vụ nuôi dưỡng, chăm sóc và giáo dục trẻ em', 'Không Phân Biệt Đối Xử', 'Bảo Vệ Quyền Lợi Hợp Pháp của Mọi Thành Viên'], 'Explain': "Câu trả lời của sinh viên đã trích xuất được nhiều từ khóa và thông tin quan trọng liên quan đến nguyên tắc cơ bản của chế độ hôn nhân và gia đình theo Luật Hôn nhân và Gia đình năm 2014. Tuy nhiên, một số nguyên tắc quan trọng như 'hôn nhân tự nguyện' và 'trách nhiệm bảo vệ trẻ em' không được đề cập. Mặc dù sinh viên đã nêu rõ các nguyên tắc như 'một vợ một chồng', 'bình đẳng giữa vợ và chồng', và 'không phân biệt đối xử', nhưng vẫn thiếu một số nội dung quan trọng từ đáp án đúng. Do đó

### Lần 1


In [75]:
len(RES_TULUAN_HYBRID)

28

In [42]:
score = 0

for i in range(len(RES_TULUAN_HYBRID)):
    if RES_TULUAN_HYBRID[i] != "Not":
        score += int(RES_TULUAN_HYBRID[i]['Score'])

score/len(RES_TULUAN_HYBRID)
#8.035714285714286

8.035714285714286

In [53]:
# Lần 2
score = 0

for i in range(len(RES_TULUAN_HYBRID)):
    if RES_TULUAN_HYBRID[i] != "Not":
        score += int(RES_TULUAN_HYBRID[i]['Score'])

score/len(RES_TULUAN_HYBRID)
#7.964285714285714

7.964285714285714

In [63]:
# Lần 3
score = 0

for i in range(len(RES_TULUAN_HYBRID)):
    if RES_TULUAN_HYBRID[i] != "Not":
        score += int(RES_TULUAN_HYBRID[i]['Score'])

score/len(RES_TULUAN_HYBRID)
# 7.928571428571429

7.928571428571429

In [77]:
# Lần 4
#RES_TULUAN_HYBRID[0]

score = 0

for i in range(len(RES_TULUAN_HYBRID)):
    if RES_TULUAN_HYBRID[i] != "Not":
        score += int(RES_TULUAN_HYBRID[i]['Score'])

score/len(RES_TULUAN_HYBRID)
# 6.678571428571429

6.678571428571429

# TULUAN_NAIVE

In [78]:
RES_TULUAN_NAIVE = []

In [79]:
RES_TULUAN_NAIVE

[]

In [80]:
for i in range(len(TULUAN_NAIVE)):

    test = PROMPT['evaluation_from_test_kw'].replace("<<q>>", TULUAN_NAIVE[i]['q']).replace("<<sga>>", TULUAN_NAIVE[i]['a']).replace("<<a>>", TULUAN_NAIVE[i]['naive'])

    res = get_valid_json_gpt40mini(test)

    print(res)

    RES_TULUAN_NAIVE.append(res)

attempt : 0
gpt-4o-mini
Định dạng JSON hợp lệ!
{'sga': ['Chế độ hôn nhân và gia đình', 'quy định của pháp luật về kết hôn, ly hôn', 'quyền và nghĩa vụ giữa vợ và chồng', 'giữa cha mẹ và con', 'cấp dưỡng', 'xác định cha, mẹ, con', 'quan hệ hôn nhân và gia đình có yếu tố nước ngoài', 'nguyên tắc cơ bản của Luật hôn nhân và gia đình năm 2000', 'hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng', 'hôn nhân giữa công dân Việt Nam thuộc các dân tộc, tôn giáo', 'gia đình ấm no, tiến bộ, hạnh phúc', 'trách nhiệm bảo vệ, hỗ trợ trẻ em, người cao tuổi, người khuyết tật', 'kế thừa, phát huy truyền thống văn hóa, đạo đức tốt đẹp'], 'anwser': ['Luật Hôn nhân và Gia đình năm 2014', 'nguyên tắc cơ bản của chế độ hôn nhân và gia đình', 'đảm bảo sự phát triển văn hóa và xã hội', 'bảo vệ quyền lợi và nghĩa vụ của các thành viên trong gia đình', 'Hôn nhân tự nguyện và bình đẳng', 'tôn trọng sự đa dạng', 'Xây dựng gia đình hạnh phúc', 'Trách nhiệm của Nhà nước và xã hội', 'Bảo tồn văn hóa'

In [81]:
len(RES_TULUAN_NAIVE)

28

In [47]:
score = 0

for i in range(len(RES_TULUAN_NAIVE)):
    if RES_TULUAN_HYBRID[i] != "Not":
        score += int(RES_TULUAN_NAIVE[i]['Score'])

score/len(RES_TULUAN_NAIVE)
#8.392857142857142

8.392857142857142

In [58]:
# Lần 2

score = 0

for i in range(len(RES_TULUAN_NAIVE)):
    if RES_TULUAN_HYBRID[i] != "Not":
        score += int(RES_TULUAN_NAIVE[i]['Score'])

score/len(RES_TULUAN_NAIVE)
# 8.321428571428571

8.321428571428571

In [68]:
# Lần 3
score = 0

for i in range(len(RES_TULUAN_NAIVE)):
    if RES_TULUAN_HYBRID[i] != "Not":
        score += int(RES_TULUAN_NAIVE[i]['Score'])

score/len(RES_TULUAN_NAIVE)

8.321428571428571

In [82]:
# Lần 4

score = 0

for i in range(len(RES_TULUAN_NAIVE)):
    if RES_TULUAN_HYBRID[i] != "Not":
        score += int(RES_TULUAN_NAIVE[i]['Score'])

score/len(RES_TULUAN_NAIVE)
# 7.214285714285714

7.214285714285714

# TUVAN_HYBRID

In [95]:
RES_TUVAN_HYBRID = []

In [96]:
RES_TUVAN_HYBRID

[]

In [86]:
TUVAN_HYBRID[13]

{'q': 'Vợ chồng em cưới nhau được 4 năm và có một người con chung 2 tuổi. Do mâu thuẫn trong việc chăm sóc con nên muốn ly hôn. Bất cứ khi nào vợ chồng em có mâu thuẫn là cô ấy lại bế con bỏ nhà đi. Mặc dù chưa ly hôn nhưng nếu vợ em đưa con em đi mà chưa được sự đồng ý của em thì ai sẽ can thiệp? Em có thể tìm sự trợ giúp từ cơ quan nào? Và vợ em sẽ phải chịu trách nhiệm như thế nào? ',
 'hybrid': '## Giải đáp về quyền nuôi con và trách nhiệm của các bên trong ly hôn\n\nTrong trường hợp của bạn, các vấn đề liên quan đến việc nuôi dưỡng cũng như quyền và nghĩa vụ của cha mẹ đối với con cái sẽ được quy định theo Luật Hôn nhân và Gia đình 2014, cụ thể là các khoản liên quan đến quyền nuôi con và nghĩa vụ của vợ chồng trong quy trình ly hôn.\n\n### 1. Quyền Nuôi Con\n\nTheo **Điều 81** của Luật Hôn nhân và Gia đình 2014:\n\n- Khi xảy ra ly hôn, vợ chồng có quyền thỏa thuận về người trực tiếp nuôi con, nếu không thỏa thuận được thì Tòa án sẽ quyết định trên cơ sở lợi ích của trẻ em.\n- Tro

In [97]:
for i in range(len(TUVAN_HYBRID)):

    test = PROMPT['evaluation_from_test'].replace("<<q>>", TUVAN_HYBRID[i]['q']).replace("<<sga>>", TUVAN_HYBRID[i]['a']).replace("<<a>>", TUVAN_HYBRID[i]['hybrid'])

    res = get_valid_json_gpt40mini(test)

    print(res)

    RES_TUVAN_HYBRID.append(res)

attempt : 0
gpt-4o-mini
Định dạng JSON hợp lệ!
{'Score': 8, 'Explain': 'Câu trả lời đã cung cấp nhiều thông tin hữu ích liên quan đến quy định về mang thai hộ và nuôi con nuôi, bao gồm các điều luật cụ thể. Tuy nhiên, một số thông tin có thể được diễn đạt ngắn gọn hơn và có thể thiếu một số chi tiết về điều kiện cụ thể để mang thai hộ theo Luật Hôn nhân và Gia Đình năm 2014. Mặc dù vậy, nội dung chính vẫn được truyền đạt rõ ràng và đúng với quy định pháp luật.'}
attempt : 0
gpt-4o-mini
Định dạng JSON hợp lệ!
{'Score': '8', 'Explain': 'Câu trả lời đã cung cấp nhiều thông tin hữu ích và chính xác liên quan đến quy định của Luật Hôn nhân và Gia đình 2014, bao gồm việc hôn nhân một vợ một chồng, hậu quả pháp lý của việc không đăng ký kết hôn, và việc kết hôn trái pháp luật. Tuy nhiên, một số thông tin có thể được diễn đạt ngắn gọn hơn và không hoàn toàn đầy đủ như trong câu trả lời đúng, ví dụ như không nhấn mạnh rõ ràng rằng việc tổ chức đám cưới không có giá trị pháp lý. Do đó, điểm số l

In [98]:
len(RES_TUVAN_HYBRID)

14

In [94]:
# Lần 1
score = 0

for i in range(len(RES_TUVAN_HYBRID)):
    if RES_TUVAN_HYBRID[i] != "Not":
        score += int(RES_TUVAN_HYBRID[i]['Score'])

score/len(RES_TUVAN_HYBRID)
# 8.142857142857142

8.142857142857142

In [99]:
# Lần 2
score = 0

for i in range(len(RES_TUVAN_HYBRID)):
    if RES_TUVAN_HYBRID[i] != "Not":
        score += int(RES_TUVAN_HYBRID[i]['Score'])

score/len(RES_TUVAN_HYBRID)
#8.0

8.0

# TUVAN_NAIVE

In [110]:
RES_TUVAN_NAIVE = []

In [111]:
RES_TUVAN_NAIVE

[]

In [103]:
TUVAN_NAIVE[13]

{'q': 'Vợ chồng em cưới nhau được 4 năm và có một người con chung 2 tuổi. Do mâu thuẫn trong việc chăm sóc con nên muốn ly hôn. Bất cứ khi nào vợ chồng em có mâu thuẫn là cô ấy lại bế con bỏ nhà đi. Mặc dù chưa ly hôn nhưng nếu vợ em đưa con em đi mà chưa được sự đồng ý của em thì ai sẽ can thiệp? Em có thể tìm sự trợ giúp từ cơ quan nào? Và vợ em sẽ phải chịu trách nhiệm như thế nào? ',
 'naive': 'Trong tình huống của bạn, việc vợ bạn tự ý dẫn con đi mà không có sự đồng ý của bạn là một vấn đề pháp lý nghiêm trọng, và bạn có quyền yêu cầu sự can thiệp từ các cơ quan chức năng. Dưới đây là một số điều luật và giải pháp mà bạn có thể tham khảo.\n\n### Quyền và Nghĩa vụ của Cha Mẹ\n\n1. **Điều 81 Luật Hôn nhân và Gia đình 2014**: Điều này xác định rằng cha mẹ có trách nhiệm nuôi dưỡng và chăm sóc con cái. Trong trường hợp có tranh chấp về quyền nuôi con, phải xem xét quyền lợi của trẻ em trước hết và ưu tiên quy định bé sống cùng với cha hoặc mẹ. Việc dẫn trẻ đi mà không có sự đồng ý từ 

In [112]:
for i in range(len(TUVAN_NAIVE)):

    test = PROMPT['evaluation_from_test'].replace("<<q>>", TUVAN_NAIVE[i]['q']).replace("<<sga>>", TUVAN_NAIVE[i]['a']).replace("<<a>>", TUVAN_NAIVE[i]['naive'])

    res = get_valid_json_gpt40mini(test)

    print(res)

    RES_TUVAN_NAIVE.append(res)

attempt : 0
gpt-4o-mini
Định dạng JSON hợp lệ!
{'Score': 9, 'Explain': 'Câu trả lời của sinh viên đã cung cấp khoảng 80-90% thông tin chính xác và hữu ích từ đáp án đúng. Sinh viên đã nêu rõ các điều khoản liên quan trong Luật Hôn nhân và Gia đình năm 2014, cụ thể là Điều 95 và Điều 94, đồng thời giải thích rõ ràng về quyền lợi của vợ chồng trong việc nhờ mang thai hộ. Tuy nhiên, một số thông tin có thể được trình bày ngắn gọn hơn và có thể có thêm một số chi tiết về điều kiện cụ thể hơn từ Điều 95 để hoàn thiện hơn nữa.'}
attempt : 0
gpt-4o-mini
Định dạng JSON hợp lệ!
{'Score': 8, 'Explain': 'Câu trả lời đã cung cấp nhiều thông tin chính xác liên quan đến quy định của Luật Hôn nhân và Gia đình 2014, đặc biệt là các điều kiện kết hôn và các trường hợp cấm kết hôn. Tuy nhiên, câu trả lời thiếu một số thông tin quan trọng về việc tổ chức đám cưới mà không đăng ký kết hôn không có giá trị pháp lý, và không đề cập đến việc chung sống như vợ chồng. Mặc dù đã nêu rõ rằng hành vi này có thể v

In [113]:
len(RES_TUVAN_NAIVE)

14

In [109]:
# Lần 1
score = 0

for i in range(len(RES_TUVAN_NAIVE)):
    if RES_TUVAN_NAIVE[i] != "Not":
        score += int(RES_TUVAN_NAIVE[i]['Score'])

score/len(RES_TUVAN_NAIVE)
# 8.357142857142858

8.357142857142858

In [114]:
# Lần 2
score = 0

for i in range(len(RES_TUVAN_NAIVE)):
    if RES_TUVAN_NAIVE[i] != "Not":
        score += int(RES_TUVAN_NAIVE[i]['Score'])

score/len(RES_TUVAN_NAIVE)
# 8.285714285714286

8.285714285714286

# Baseline GPT-4o

In [116]:
for i in TULUAN_HYBRID:
    q = i['q']

    res = get_completion(q, model = 'gpt-4o')

    i['baseline'] = res

gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o
gpt-4o


In [117]:
TULUAN_HYBRID[0]

{'q': 'Theo Luật hôn nhân và gia đình năm 2014, những nguyên tắc cơ bản của chế độ hôn nhân và gia đình ở nước ta được quy định như thế nào?',
 'hybrid': '## Nguyên Tắc Cơ Bản của Chế Độ Hôn Nhân và Gia Đình theo Luật Hôn Nhân và Gia Đình Năm 2014\n\nLuật Hôn nhân và Gia đình năm 2014, có hiệu lực từ ngày 01/01/2015, đã quy định một cách rõ ràng các nguyên tắc cơ bản trong chế độ hôn nhân và gia đình tại Việt Nam. Những nguyên tắc này không chỉ bảo đảm quyền và nghĩa vụ của các bên trong quan hệ hôn nhân mà còn thể hiện sự bình đẳng và công bằng giữa các thành viên trong gia đình. Dưới đây là một số nguyên tắc cơ bản được quy định trong bộ luật này.\n\n### 1. Nguyên Tắc Một Vợ Một Chồng\n\nLuật quy định rằng chế độ hôn nhân tại Việt Nam là hôn nhân một vợ một chồng. Điều này nhấn mạnh rằng mỗi người chỉ có thể kết hôn với một người khác tại cùng một thời điểm, nhằm bảo vệ quyền lợi hợp pháp và đạo đức trong quan hệ hôn nhân.\n\n### 2. Nguyên Tắc Bình Đẳng Giữa Vợ và Chồng\n\nLuật Hôn n

In [127]:
TULUAN_GPT = []

In [128]:
for i in range(len(TULUAN_HYBRID)):

    test = PROMPT['evaluation_from_test_kw'].replace("<<q>>", TULUAN_HYBRID[i]['q']).replace("<<sga>>", TULUAN_HYBRID[i]['a']).replace("<<a>>", TULUAN_HYBRID[i]['baseline'])

    res = get_valid_json_gpt40mini(test)

    print(res)

    TULUAN_GPT.append(res)

attempt : 0
gpt-4o-mini
Định dạng JSON hợp lệ!
{'sga': ['Chế độ hôn nhân và gia đình', 'quy định của pháp luật', 'kết hôn, ly hôn', 'quyền và nghĩa vụ giữa vợ và chồng', 'giữa cha mẹ và con', 'cấp dưỡng', 'xác định cha, mẹ, con', 'quan hệ hôn nhân và gia đình có yếu tố nước ngoài', 'nguyên tắc cơ bản', 'hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng', 'bảo vệ quyền lợi của phụ nữ và trẻ em', 'trách nhiệm bảo vệ, hỗ trợ trẻ em, người cao tuổi, người khuyết tật', 'kế thừa, phát huy truyền thống văn hóa, đạo đức tốt đẹp'], 'anwser': ['Tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng', 'Bảo vệ quyền lợi của phụ nữ và trẻ em', 'Bảo vệ chế độ hôn nhân và gia đình', 'Kế thừa và phát huy truyền thống văn hóa tốt đẹp của dân tộc', 'Bình đẳng giới', 'Tôn trọng quyền tự do tín ngưỡng, tôn giáo'], 'Explain': "Câu trả lời của sinh viên đã trích xuất được nhiều từ khóa và thông tin quan trọng từ đáp án đúng, nhưng vẫn thiếu một số nội dung quan trọng như 'cấp dưỡng', 'xác 

In [125]:
len(TULUAN_GPT)

28

In [122]:
score = 0

for i in range(len(TULUAN_GPT)):
    if TULUAN_GPT[i] != "Not":
        score += int(TULUAN_GPT[i]['Score'])

score/len(TULUAN_GPT)
#8.035714285714286

8.035714285714286

In [126]:
# Lần 2

score = 0

for i in range(len(TULUAN_GPT)):
    if TULUAN_GPT[i] != "Not":
        score += int(TULUAN_GPT[i]['Score'])

score/len(TULUAN_GPT)
# 8.0

8.0

In [129]:
# Lần 3

score = 0

for i in range(len(TULUAN_GPT)):
    if TULUAN_GPT[i] != "Not":
        score += int(TULUAN_GPT[i]['Score'])

score/len(TULUAN_GPT)
# 6.964285714285714

6.964285714285714

# Baseline GPT-4o-mini

In [130]:
for i in TULUAN_HYBRID:
    q = i['q']

    res = get_completion(q, model = 'gpt-4o-mini')

    i['mini'] = res

gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini
gpt-4o-mini


In [133]:
TULUAN_HYBRID

[{'q': 'Theo Luật hôn nhân và gia đình năm 2014, những nguyên tắc cơ bản của chế độ hôn nhân và gia đình ở nước ta được quy định như thế nào?',
  'hybrid': '## Nguyên Tắc Cơ Bản của Chế Độ Hôn Nhân và Gia Đình theo Luật Hôn Nhân và Gia Đình Năm 2014\n\nLuật Hôn nhân và Gia đình năm 2014, có hiệu lực từ ngày 01/01/2015, đã quy định một cách rõ ràng các nguyên tắc cơ bản trong chế độ hôn nhân và gia đình tại Việt Nam. Những nguyên tắc này không chỉ bảo đảm quyền và nghĩa vụ của các bên trong quan hệ hôn nhân mà còn thể hiện sự bình đẳng và công bằng giữa các thành viên trong gia đình. Dưới đây là một số nguyên tắc cơ bản được quy định trong bộ luật này.\n\n### 1. Nguyên Tắc Một Vợ Một Chồng\n\nLuật quy định rằng chế độ hôn nhân tại Việt Nam là hôn nhân một vợ một chồng. Điều này nhấn mạnh rằng mỗi người chỉ có thể kết hôn với một người khác tại cùng một thời điểm, nhằm bảo vệ quyền lợi hợp pháp và đạo đức trong quan hệ hôn nhân.\n\n### 2. Nguyên Tắc Bình Đẳng Giữa Vợ và Chồng\n\nLuật Hôn

In [137]:
TULUAN_GPT_MINI = []

In [138]:
for i in range(len(TULUAN_HYBRID)):

    test = PROMPT['evaluation_from_test_kw'].replace("<<q>>", TULUAN_HYBRID[i]['q']).replace("<<sga>>", TULUAN_HYBRID[i]['a']).replace("<<a>>", TULUAN_HYBRID[i]['mini'])

    res = get_valid_json_gpt40mini(test)

    print(res)

    TULUAN_GPT_MINI.append(res)

attempt : 0
gpt-4o-mini
Định dạng JSON hợp lệ!
{'sga': ['hôn nhân tự nguyện', 'một vợ một chồng', 'bình đẳng vợ chồng', 'hôn nhân giữa công dân Việt Nam', 'gia đình ấm no, tiến bộ, hạnh phúc', 'trách nhiệm bảo vệ trẻ em, người cao tuổi, người khuyết tật', 'kế thừa truyền thống văn hóa, đạo đức'], 'anwser': ['bình đẳng giới', 'tự nguyện', 'một vợ, một chồng', 'gia đình là tế bào của xã hội', 'bảo vệ quyền và lợi ích hợp pháp của các thành viên', 'tôn trọng và bảo vệ quyền tự do, quyền con người'], 'Explain': 'Câu trả lời của sinh viên đã trích xuất được một số từ khóa và thông tin quan trọng liên quan đến nguyên tắc cơ bản của chế độ hôn nhân và gia đình theo Luật hôn nhân và gia đình năm 2014. Tuy nhiên, một số thông tin quan trọng như trách nhiệm bảo vệ trẻ em, người cao tuổi, người khuyết tật và kế thừa truyền thống văn hóa, đạo đức không được đề cập. Mặc dù có một số điểm tương đồng về nội dung, nhưng câu trả lời vẫn thiếu sót và không hoàn toàn chính xác so với đáp án đúng.', 'Scor

In [135]:
len(TULUAN_GPT_MINI)

28

In [136]:
# Lần 1

score = 0

for i in range(len(TULUAN_GPT_MINI)):
    if TULUAN_GPT_MINI[i] != "Not":
        score += int(TULUAN_GPT_MINI[i]['Score'])

score/len(TULUAN_GPT_MINI)
# 7.285714285714286

7.285714285714286

In [139]:
# Lần 2 kw

score = 0

for i in range(len(TULUAN_GPT_MINI)):
    if TULUAN_GPT_MINI[i] != "Not":
        score += int(TULUAN_GPT_MINI[i]['Score'])

score/len(TULUAN_GPT_MINI)
# 6.25

6.25